In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [2]:
# CIFAR 10 verisetini yükleyelim
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

In [3]:
# Sınıf etiketlerine one-hot-encoding işlemi uygulayalım
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

In [4]:
model = Sequential()

# 32 adet 5x5 boyutunda filtreye sahip, ReLU aktivasyonlu CONV katmanı
model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), padding='SAME'))
model.add(Activation('relu'))

# 32 adet 5x5 boyutunda filtreye sahip, ReLU aktivasyonlu CONV katmanı
model.add(Conv2D(32, (5, 5), padding='SAME'))
model.add(Activation('relu'))

# 2x2 boyutunda filtreye sahip MAX POOL katmanı
model.add(MaxPooling2D(pool_size=(2, 2), padding='SAME'))

# %25 atılsın
model.add(Dropout(0.25))

# Vektörize edelim
model.add(Flatten())

# 32 nöronlu ReLU aktivasyonlu katman
model.add(Dense(32))
model.add(Activation('relu'))

# Son katman: 10 nöronlu Softmax aktivasyonlu katman
model.add(Dense(10))
model.add(Activation('softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        25632     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
__________

In [6]:
# Compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(), 
              metrics=['accuracy'])

In [7]:
# Veri artırma yöntemlerini (kaydırma, dikey ve yatay simetrik alma) tanımla
datagen = ImageDataGenerator(width_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)

# Veri akışını tanımla
train_generator = datagen.flow(X_train, Y_train, batch_size=64)

In [8]:
# MODEL EĞİT
model.fit_generator(train_generator, 
                    steps_per_epoch=50000//64, 
                    epochs=1, 
                    validation_data=(X_test, Y_test),
                    validation_steps=10000//64)

Epoch 1/1
781/781 [==============================] - 159s - loss: 1.7933 - acc: 0.3374 - val_loss: 1.5638 - val_acc: 0.4385
